In [7]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_curve, roc_auc_score, auc, precision_score, classification_report, confusion_matrix, accuracy_score, recall_score, f1_score
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import numpy as np
import scipy.io as sp
from sklearn.decomposition import PCA
from sklearn.linear_model import Perceptron
from sklearn.calibration import CalibratedClassifierCV
import xlsxwriter
import pandas as pd
from IPython.display import display
from sklearn.model_selection import StratifiedKFold




#Traitement du fichier .mat
cancercell = sp.loadmat('breastw.mat')
X = cancercell['X']
y = cancercell['y']
#print(cancercell)
y=np.reshape(y,(683,))
#Séparation des données d'apprentissage et des données tests
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, stratify=y)
#fin traitement mat

"""
PP = Perceptron(penalty='elasticnet') #random_state=42
PP.fit(X_train, y_train)
predictions_test = PP.predict(X_test)

#Accuracy
result1 = accuracy_score(y_test,predictions_test)
print("Accuracy 1 :",result1)

#ROC curve
clf_isotonic = CalibratedClassifierCV(PP, cv='prefit', method='isotonic') #il faut encore fait la validation croisée au début (équilibrer notre dataset)
#CV correspond à la validation croisée, si dataset déjà rééquilibré : cv = 'predict'
clf_isotonic.fit(X_test, y_test)
preds = clf_isotonic.predict_proba(X_test)

#y_scores1 = PP.predict_proba(X_test) #il ne connaît pas predict_proba
fpr, tpr, threshold = roc_curve(y_test, preds[:, 1])
roc_auc = auc(fpr, tpr)
print("AUC 1 : ", roc_auc)

plt.title('Receiver Operating Characteristic')
plt.plot(fpr, tpr, 'b', label = 'AUC 1 = %0.2f' % roc_auc)
plt.legend(loc = 'lower right')
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0, 1])
plt.ylim([0, 1])
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.title('ROC Curve Simple Perceptron')
plt.show()


def create(row, col):
    return [[[]for _ in range(col)]  for _ in range(row)]

penalty = ['l1', 'l2','elasticnet','None']
Max_iter = []
for k in range (1,15):
    Max_iter.append(50*k)
Eta =[]
for k in range (-5,3):
    Eta.append(10**(-k)) 
Alpha = []
for k in range (0,5):
    Alpha.append(10**(-k))

MC = create(len(penalty)*len(Max_iter)*,3)
MC[0][0]='penalty'
MC[0][1]='AUC'
MC[0][2]='Accuracy'
i=1

for k in penalty:
    
    MC[i][0]= k

    PP = Perceptron(penalty=k) #random_state=42
    PP.fit(X_train, y_train)
    predictions_test = PP.predict(X_test)

        
    #ROC curve
    clf_isotonic = CalibratedClassifierCV(PP, cv='prefit', method='isotonic') #il faut encore fait la validation croisée au début (équilibrer notre dataset)
    #CV correspond à la validation croisée, si dataset déjà rééquilibré : cv = 'predict'
    clf_isotonic.fit(X_test, y_test)
    preds = clf_isotonic.predict_proba(X_test)

    #y_scores1 = PP.predict_proba(X_test) #il ne connaît pas predict_proba
    fpr, tpr, threshold = roc_curve(y_test, preds[:, 1])
    roc_auc = auc(fpr, tpr)
    
    #Accuracy
    result1 = accuracy_score(y_test,predictions_test)
    
    MC[i][1]=(round(roc_auc, 6))
    MC[i][2]= result1
            
    i += 1 

print(MC)
workbook = xlsxwriter.Workbook('Comparaison matrice Perceptron simple.xlsx')
worksheet = workbook.add_worksheet()
col = 0
for row, data in enumerate(MC):
    worksheet.write_row(row, col, data)
workbook.close()

"""



FileNotFoundError: [Errno 2] No such file or directory: 'breastw.mat'

DATA FRAME

In [ ]:
penalty = ['l1', 'l2','elasticnet','None']
Max_iter = []
for k in range (1,15):
    Max_iter.append(50*k)
Eta =[]
for k in range (-5,3):
    Eta.append(10**(-k)) 
Alpha = []
for k in range (0,5):
    Alpha.append(10**(-k))

#data = np.array([['penalty','Max_iter','Eta','Alpha','F1 score','recall','precision','accuracy','AUC',]])
skf = StratifiedKFold(n_splits=10,shuffle=True) # équilibrage dataset

for train_index, test_index in skf.split(X,y):
    
    test =False
    X_train, X_test = X[train_index,:], X[test_index,:]
    y_train, y_test = y[train_index], y[test_index]
    y_train=np.ravel(y_train)
    y_test=np.ravel(y_test)

    #Méthode Perceptron simple

    test =False
    
    for k in penalty :
        for i in Max_iter :
            for j in Eta :
                for p in Alpha : 
                    
                    #algo Perceptron
                    model_Percep = Perceptron(penalty=k, max_iter=i, eta0=j, alpha=p)
                    model_Percep.fit(X_train,y_train)
                    y_pred_percep = model_Percep.predict(X_test)
                    
                    #fin algo Perceptron
                
                    
                    #ROC curve
                    clf_isotonic = CalibratedClassifierCV(model_Percep, cv='prefit', method='isotonic')
                    #CV correspond à la validation croisée, si dataset déjà rééquilibré : cv = 'predict'
                    clf_isotonic.fit(X_test, y_test)
                    y_scores_Percep = clf_isotonic.predict_proba(X_test)
    
                    y_scores_Percep = y_scores_Percep[:, 1]
                    
                    F1_Percep = round(f1_score(y_test,y_pred_percep),3)
                    recall_Percep = round(recall_score(y_test,y_pred_percep),3)
                    precision_Percep = round(precision_score(y_test,y_pred_percep,zero_division=0),3)
                    accuracy_Percep = round(accuracy_score(y_test,y_pred_percep),3)
                    AUC_Percep = round(roc_auc_score(y_test, y_scores_Percep),3)
                    data = np.append(data, [[k,i,j,p,F1_Percep,recall_Percep,precision_Percep,accuracy_Percep,AUC_Percep]],axis=0)
                    
                    if test == False :
                        matrice_Percep = np.array([[k,i,j,p,F1_Percep,recall_Percep,precision_Percep,accuracy_Percep,AUC_Percep]])
                        test = True
                    else :
                        matrice_Percep = np.append(matrice_Percep,np.array([k,i,j,p,F1_Percep,recall_Percep,precision_Percep,accuracy_Percep,AUC_Percep]).reshape(1,9),axis=0)
        
            #Fin méthode Percep



df = pd.DataFrame(matrice_Percep, columns=['penalty','Max_iter','Eta','Alpha','F1 score','Recall','Precision','Accuracy','AUC'])
#df=df.drop(df.index[0])
df = df.astype({'F1 score': float, 'Recall': float,'Precision': float,'Accuracy': float,'AUC': float})
filtered_values_1 = np.where((df['F1 score']>=0.96) & (df['Recall']>=0.925) & (df['Precision']>=0.925) & (df['AUC']>=0.925))
display(df.iloc[filtered_values_1].sort_values(by=['F1 score'], ascending=False))


NameError: name 'np' is not defined

In [ ]:
data_csv_Perceptron = df.to_csv('data_csv_Perceptron.csv')

In [ ]:
classdata=pd.read_csv('data_csv_Perceptron.csv')
classdata.mean()


/var/folders/s6/p3txrvvd4498vs_xw7ryvx7h0000gn/T/ipykernel_71330/112100171.py:2: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  classdata.mean()


Unnamed: 0    11200.500000
Max_iter        375.000000
Eta           13888.888750
Alpha             0.222220
F1 score          0.666624
recall            0.699216
precision         0.664886
accuracy          0.842228
AUC               0.876559
dtype: float64